# PIP

In [ ]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
!pip install python-binance
!pip install config
!pip install six
!pip install websocket-client

In [3]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 160690 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 12.1MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp37-cp37m-linux_x86_64.whl size=1438010 sha256=b1c475137d9660b523718289ac3f264304e15187bdaa62c12ef0c5b8ee16f706
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib


In [ ]:
!pip install python-binance

In [14]:
!pip install config

In [ ]:
!pip install six

In [ ]:
pip install websocket-client

In [ ]:
!pip install ccxt

# RSI MODEL

In [ ]:
#https://github.com/hackingthemarkets/binance-tutorials/blob/master/rsibot/bot.py

import websocket, json, pprint, talib, numpy
import config
from binance.client import Client
from binance.enums import *

SOCKET = "wss://stream.binance.com:9443/ws/ethusdt@kline_1m"

RSI_PERIOD = 2
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
TRADE_SYMBOL = 'ETHUSD'
TRADE_QUANTITY = 0.05

closes = []
in_position = False
config.API_KEY = '651723883213a97070e71139a9803b3071c207ced51ffd2489e3dbb635e502df'
config.API_SECRET = '6ff56e681ced840c886e2d5bdca27d2e4e136d2b1ca7f3c7083a61e090c740c0'

client = Client(config.API_KEY, config.API_SECRET, tld='com',testnet = 'true')

def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True

    
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    print('received message')
    json_message = json.loads(message)
    # pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        print(closes)

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes)
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            print("all rsis calculated so far")
            print(rsi)
            last_rsi = rsi[-1]
            print("the current rsi is {}".format(last_rsi))

            if last_rsi > RSI_OVERBOUGHT:
                if in_position:
                    print("Overbought! Sell! Sell! Sell!")
                    # put binance sell logic here
                    order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
                    if order_succeeded:
                        in_position = False
                else:
                    print("It is overbought, but we don't own any. Nothing to do.")
            
            if last_rsi < RSI_OVERSOLD:
                if in_position:
                    print("It is oversold, but you already own it, nothing to do.")
                else:
                    print("Oversold! Buy! Buy! Buy!")
                    # put binance buy order logic here
                    order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                    if order_succeeded:
                        in_position = True

                
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

# ORDER TEST

In [4]:
import websocket, json, pprint, talib, numpy

SOCKET = "wss://stream.binance.com:9443/ws/ethusdt@kline_1m"

import ccxt 
import pprint


# 파일로부터 apiKey, Secret 읽기 

api_key = '651723883213a97070e71139a9803b3071c207ced51ffd2489e3dbb635e502df'

secret = '6ff56e681ced840c886e2d5bdca27d2e4e136d2b1ca7f3c7083a61e090c740c0'

# binance 객체 생성
exchange = ccxt.binance(config={
    'apiKey': api_key,
    'secret': secret,
    'enableRateLimit': True, 
    'options': {
      'defaultType': 'future'
    },
})
exchange.set_sandbox_mode(True)
balance = exchange.fetch_balance()
print(balance['USDT'])
# closes = []
in_position = False
# order = binance.create_limit_buy_order(
#     symbol='BTC/USDT', 
#     amount=0.003, 
#     price=50000, 
#     params={'type': 'future'}
# )

# pprint.pprint(order)
exchange.create_market_sell_order('BTC/USDT',0.003)
# markets = exchange.load_markets()
# positions = exchange.fetch_positions()

# pprint.pprint(exchange.fapiPrivate_get_positionrisk())
a = exchange.fapiPrivate_get_positionrisk()
b = next(item for item in a if item['symbol'] == 'BTCUSDT')
print(b)

{'free': 91091.21045006, 'used': 0.0, 'total': 91091.21045006}
{'symbol': 'BTCUSDT', 'positionAmt': '-0.003', 'entryPrice': '57092.43000', 'markPrice': '57099.47942601', 'unRealizedProfit': '-0.02114827', 'liquidationPrice': '113716.08889774', 'leverage': '1', 'maxNotionalValue': '9.223372036854776E18', 'marginType': 'isolated', 'isolatedMargin': '171.21442149', 'isAutoAddMargin': 'false', 'positionSide': 'BOTH', 'notional': '-171.29843827', 'isolatedWallet': '171.23556976', 'updateTime': '1620083824501'}


# MAKE RSI MODEL TO CCXT



In [19]:
import websocket, json, pprint, talib, numpy

SOCKET = "wss://stream.binance.com:9443/ws/ethusdt@kline_1m"
api_key = '651723883213a97070e71139a9803b3071c207ced51ffd2489e3dbb635e502df'

secret = '6ff56e681ced840c886e2d5bdca27d2e4e136d2b1ca7f3c7083a61e090c740c0'
RSI_PERIOD = 2
RSI_OVERBOUGHT = 120
RSI_OVERSOLD = 30
TRADE_SYMBOL = 'ETHUSD'
TRADE_QUANTITY = 0.05
# binance 객체 생성
exchange = ccxt.binance(config={
    'apiKey': api_key,
    'secret': secret,
    'enableRateLimit': True, 
    'options': {
      'defaultType': 'future'
    },
})
exchange.set_sandbox_mode(True)
balance = exchange.fetch_balance()
print(balance['USDT'])

in_position = False
closes = []

def buy(self, amount):
  exchange.create_market_buy_order('BTC/USDT',amount)

def sell(self, amount):
  exchange.create_market_sell_order('BTC/USDT',amount)

def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    print('received message')
    json_message = json.loads(message)
    # pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        print(closes)

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes)
            print(np_closes)
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            print("all rsis calculated so far")
            print(rsi)
            last_rsi = rsi[-1]
            print("the current rsi is {}".format(last_rsi))

            if last_rsi > RSI_OVERBOUGHT:
                if in_position:
                    print("Overbought! Sell! Sell! Sell!")
                    # put binance sell logic here
                    exchange.create_market_sell_order('BTC/USDT',0.03)
                    if order_succeeded:
                        in_position = False
                else:
                    print("It is overbought, but we don't own any. Nothing to do.")
            
            if last_rsi < RSI_OVERSOLD:
                if in_position:
                    print("It is oversold, but you already own it, nothing to do.")
                else:
                    print("Oversold! Buy! Buy! Buy!")
                    # put binance buy order logic here
                    exchange.create_market_buy_order('BTC/USDT',0.03)
                    if order_succeeded:
                        in_position = True
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

{'free': 91091.18966471, 'used': 0.0, 'total': 91091.18966471}
opened connection
received message
received message
received message
received message
received message
received message
received message
received message
received message
candle closed at 3442.58000000
closes
[3442.58]
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
received message
candle closed at 3432.97000000
closes
[3442.58, 3432.97]
received message
received message
received message
received message
received message
received message
received message
received message
received messag

False